<a href="https://colab.research.google.com/github/joserafaelrebelo/pdm2023/blob/dev/kaggle_experiments/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## imports and stuff

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from tqdm import tqdm
import torch
import os
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## finetunned model

In [ ]:
label2id = {
    "Muito Negativo": 0,
    "Negativo": 1,
    "Neutro": 2,
    "Positivo": 3,
    "Muito Positivo": 4
}

In [ ]:
# get model
model_id = "neuralmind/bert-base-portuguese-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_id, label2id=label2id)

tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

csv_file_path = "/content/drive/MyDrive/4° Semestre/PDM/reviews_test_kaggle.csv"
csv_file = '/content/drive/MyDrive/4° Semestre/PDM/bert_reviews.csv'

if not os.path.exists(csv_file):
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['id', 'sentiment'])
    print("Arquivo CSV criado com sucesso.")

with open(csv_file_path, 'r') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    comment_data = list(csv_reader)

In [ ]:
for row in tqdm(comment_data):
  content = row['content']
  if len(content) > 512:
    content = content[:512]
  classification = pipe(content)
  label = classification[0]['label']
  with open(csv_file, 'a', newline='', encoding='utf-8') as file:
      writer = csv.writer(file)
      writer.writerow([row['id'], label2id.get(label)])

del model, pipe
torch.cuda.empty_cache()

100%|██████████| 57598/57598 [3:04:10<00:00,  5.21it/s]


In [ ]:
with open(csv_file, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)

    # Use len() to get the number of rows in the CSV file
    csv_length = len(list(csv_reader))
    csv_list = list(csv_reader)

print(csv_length)

115197


In [ ]:
import pandas as pd